# S3. Fine-tuning de modelo pre-entrenado para la clasificación de PAJAROS


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
train_data, test_data = tfds.load('caltech_birds2011', split=['train', 'test'], as_supervised=True)
train_size = len(train_data)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

...

Generating train examples...:   0%|          | 0/5994 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/caltech_birds2011/0.1.1.incomplete8PRULU/caltech_birds2011-train.tfrecord*…

Generating test examples...:   0%|          | 0/5794 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/caltech_birds2011/0.1.1.incomplete8PRULU/caltech_birds2011-test.tfrecord*.…

Dataset caltech_birds2011 downloaded and prepared to /root/tensorflow_datasets/caltech_birds2011/0.1.1. Subsequent calls will reuse this data.


### Preproceso

Primero definimos una función que aplica el preproceso necesario a una muestra de entrenamiento (imagen, etiqueta de clase). Esto incluye redimensionar la imagen a 224 x 224, el preproceso específico de la red ResNet50 y convertir la etiqueta de clase a one-hot encoding.

In [ ]:
from keras.applications.resnet50 import preprocess_input

img_size = (224, 224)
num_classes = 200

def preprocess(image, label):
    image = tf.image.resize(image, img_size) #resize al tamaño que requiere la red
    image = tf.cast(image, tf.float32) # casteo a float
    image = preprocess_input(image) # preproceso para el input requerido por la red, la red viene con su funcion de preoproceso
    label = tf.one_hot(label, num_classes) # convertimos etiqueta en un vector one-hot
    return image, label

A continuación indicamos que la función anteriormente definida se aplicará a cada imagen cuando sea necesario. Para ello utilizamos la función [map()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map) del objeto [Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) y le pasamos como parámetro la función de preproceso que queremos que sea aplique a cada muestra del conjunto de datos.

In [ ]:
train_data = train_data.map(preprocess) #map lo hará bajo demanda de datos, pues hacer esto offline ocupa 40GB!!!
test_data = test_data.map(preprocess)

Las funciones [take()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#take) y [skip()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#skip) combinadas permiten definir los conjuntos de entrenamiento y validación como nuevos Datasets.

In [ ]:
train_size = int(0.8 * train_size)
train_dataset = train_data.take(train_size) # coge el 80% de los datos de entrenamiento
val_dataset = train_data.skip(train_size)   # nos quedamos con lo que quede 20%
test_dataset = test_data

#take y skip para mantener siempre los datos dentro de un objeto DataSet

print(len(train_dataset),len(val_dataset))

4795 1199


### Carga del modelo pre-entrenado

Seguidamente procedemos con la carga del modelo Inception V3 con los pesos resultantes de entrenarlo con la base de datos Imagenet, pero no queremos que el modelo incluya la capa de salida (include_top=False) que por defecto es una softmax de 1000 clases.

In [ ]:
from keras.applications.resnet50 import ResNet50

# (3,) es por tres canales, quitar las cabeza del modelo (capa densa de salida) y carga los pesos entrenados con imagenet
model = ResNet50(input_shape=img_size + (3,),include_top=False, weights='imagenet')
model.summary()

94765736/94765736 [==============================] - 3s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                      

### Preparación del modelo pre-entrenado

Vamos a preparar la red Inception V3 para ser entrenada (fine-tuning) con CIFAR-10. Dado el número de parámetros de este modelo (21M), nos limitaremos a utilizarlo con los valores por defecto y añadiremos una capa GlobalAveragePooling + MLP seguida de una softmax de 10 neuronas (10 clases) acorde a CIFAR-10 que sí que entrenaremos.

In [ ]:
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model

#no entrenes las capas del modelo preentrenado, deja los pesos tal cual
for layer in model.layers:
    layer.trainable = False

#la salida del modelo se la pasamos a pooling
x = GlobalAveragePooling2D()(model.output)
#capa densa de 1024 neuronas
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
#capa densa con activación softmax para output de clase
output = Dense(num_classes, activation='softmax')(x)

#así nuestro modelo es: preentenado(sin tocarlo) + parte final nuestra
model = Model(inputs=model.input, outputs=output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

Compilamos el modelo con los mismos parámetros que en sesiones anteriores.

In [ ]:
from keras.optimizers import Adam

opt=Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

Entrenamos el modelo utilizando los conjuntos de datos organizado en batches y que son cargados en memoria dinámicamente.

### Aumento de datos

En las sesiones anteriores, hemos utilizado la función [ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) para realizar el aumento de datos. Sin embargo, esta función no se recomienda para los nuevos desarrollos de código por estar obsoleta, y en su lugar se deben utilizar las [capas de preproceso](https://www.tensorflow.org/guide/keras/preprocessing_layers). Más concretamente, utilizaremos algunas de las [capas de preproceso de aumento de datos para imágenes](https://www.tensorflow.org/guide/keras/preprocessing_layers#image_data_augmentation).

Añadiremos estas capas de aumento de datos antes del modelo Inception V3.

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, RandomRotation, RandomTranslation, RandomZoom
from keras.models import Model

input_layer = Input(shape=img_size + (3,))

x = RandomRotation(factor=0.1, fill_mode='nearest')(input_layer)
x = RandomTranslation(height_factor=0.1, width_factor=0.1, fill_mode='nearest')(x)
x = RandomZoom(height_factor=0.2, fill_mode='nearest')(x)

res_net50_model = ResNet50(input_shape=img_size + (3,),include_top=False, weights='imagenet')

for layer in res_net50_model.layers:
    layer.trainable = False

x = res_net50_model(x)

x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

aug_model = Model(inputs=input_layer, outputs=output)

aug_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 random_rotation (RandomRot  (None, 224, 224, 3)       0         
 ation)                                                          
                                                                 
 random_translation (Random  (None, 224, 224, 3)       0         
 Translation)                                                    
                                                                 
 random_zoom (RandomZoom)    (None, 224, 224, 3)       0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_1  (None, 2048)              0   

In [ ]:
from keras.optimizers import Adam

opt=Adam(learning_rate=0.001)
aug_model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

In [ ]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.models import load_model

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=10
batch_size=32
train_dataset_batched = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset_batched = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
history = aug_model.fit(train_dataset_batched,
                    epochs=epochs,
                    verbose=1,
                    validation_data=val_dataset_batched,
                    callbacks=[reduce_lr,checkpoint])

Epoch 1/10
150/150 [==============================] - ETA: 0s - loss: 4.9164 - accuracy: 0.0551
Epoch 1: val_accuracy improved from -inf to 0.16264, saving model to best_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


150/150 [==============================] - 58s 366ms/step - loss: 4.9164 - accuracy: 0.0551 - val_loss: 3.6885 - val_accuracy: 0.1626 - lr: 0.0010
Epoch 2/10
150/150 [==============================] - ETA: 0s - loss: 3.4618 - accuracy: 0.1867
Epoch 2: val_accuracy improved from 0.16264 to 0.34862, saving model to best_model.h5
150/150 [==============================] - 55s 366ms/step - loss: 3.4618 - accuracy: 0.1867 - val_loss: 2.5995 - val_accuracy: 0.3486 - lr: 0.0010
Epoch 3/10
150/150 [==============================] - ETA: 0s - loss: 2.8071 - accuracy: 0.2891
Epoch 3: val_accuracy improved from 0.34862 to 0.40367, saving model to best_model.h5
150/150 [==============================] - 53s 357ms/step - loss: 2.8071 - accuracy: 0.2891 - val_loss: 2.2249 - val_accuracy: 0.4037 - lr: 0.0010
Epoch 4/10
150/150 [==============================] - ETA: 0s - loss: 2.4168 - accuracy: 0.3629
Epoch 4: val_accuracy improved from 0.40367 to 0.43953, saving model to best_model.h5
150/150 [====

In [ ]:
aug_model = load_model('best_model.h5')
test_dataset_batched = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
score = aug_model.evaluate(test_dataset_batched, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

Test loss: 170.55
Test accuracy: 52.52
